In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from openai import OpenAI

In [3]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [5]:
openai = OpenAI()

In [7]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


ed = Website('https://edwarddonner.com')
print(ed.title)
print(ed.text[:500])  # Print the first 500 characters of the text

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to


In [9]:
system_prompt = "You are a helpful assistant. You will be given a website URL and its content. Your task is to summarize the content of the website in a concise manner."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The content of the webiste is as follows:: \
                    Please provide a short sumry of this website in markdown. \
                    If it includes news or announcements, them summarize them too.\n\n"
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(ed))

You are looking at a website titled Home - Edward DonnerThe content of the webiste is as follows::                     Please provide a short sumry of this website in markdown.                     If it includes news or announcements, them summarize them too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder 

In [10]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

messages_for(ed)

[{'role': 'system',
  'content': 'You are a helpful assistant. You will be given a website URL and its content. Your task is to summarize the content of the website in a concise manner.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward DonnerThe content of the webiste is as follows::                     Please provide a short sumry of this website in markdown.                     If it includes news or announcements, them summarize them too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a

In [11]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content


In [12]:
summarize("https://edwarddonner.com")

'# Summary of the Website: Edward Donner\n\nEdward Donner\'s website is a personal platform where he shares his interests, projects, and professional background. Ed is deeply involved in coding and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on using AI to enhance talent discovery and management. Previously, he founded an AI startup called untapt, which was acquired in 2021.\n\n## Key Features:\n- **Connect Four**: A platform designed to challenge LLMs in diplomacy and strategic gameplay.\n- **About Ed**: A brief introduction to his interests, including coding, electronic music, and engagement with the tech community.\n- **Professional Experience**: Details his role at Nebula.io and other ventures in AI.\n- **Contact Information**: Options to connect via email and social media.\n\n## Recent Announcements:\n- **May 28, 2025**: Launching courses aimed at becoming an expert and leader in LLM technology.\n- **May 18, 2025**:

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(f"### Summary of {url}\n\n{summary}"))
display_summary("https://edwarddonner.com")

### Summary of https://edwarddonner.com

# Summary of Edward Donner's Website

Edward Donner's website provides insights into his work with large language models (LLMs), as well as his background in technology and AI. He is the co-founder and CTO of Nebula.io, a company focused on the transformative potential of AI in talent discovery and management. The site also features his interests such as coding, DJing, and electronic music production, and includes links to social media platforms.

## Recent Announcements
- **May 28, 2025:** Launch of a course to become an LLM expert and leader.
- **May 18, 2025:** Announcement of the 2025 AI Executive Briefing.
- **April 21, 2025:** Introduction of the Complete Agentic AI Engineering Course.
- **January 23, 2025:** Details about an LLM Workshop focusing on hands-on with agents.

In [14]:
display_summary("https://cnn.com")

### Summary of https://cnn.com

# Summary of CNN Website

CNN provides breaking news, latest updates, and a wide variety of content, including politics, business, health, entertainment, sports, science, and climate news. The website is organized into sections that offer comprehensive reporting on critical global issues such as the Ukraine-Russia War and the Israel-Hamas conflict.

### Key Features
- **Live Updates**: Ongoing coverage on significant events, such as mass shootings and conflicts.
- **Sections for Various Topics**: Categories include US news, world news, politics, health, and entertainment.
- **Multimedia Content**: Access to live TV, podcasts, and videos.
- **Interactive Games**: Features like crosswords and quizzes.
- **Feedback Options**: Users can provide feedback on ads and technical issues.

### Highlights from Recent Articles
- **New York City Shooting**: A gunman killed four people in Midtown, prompting ongoing investigations into the incident.
- **Gaza Crisis**: Reports indicate a severe humanitarian crisis in Gaza with over 60,000 fatalities since the start of the conflict.
- **Beijing Flooding**: Extreme weather has led to significant flooding in Beijing, causing considerable damage and casualties.
- **Alzheimer’s Research**: A study suggests lifestyle changes may help combat early-stage Alzheimer's.

For in-depth coverage and updates on current events, users can visit [CNN's website](https://www.cnn.com).

In [15]:
display_summary("https://anthropic.com")

### Summary of https://anthropic.com

# Summary of Anthropic Website

The **Anthropic** website focuses on its AI product **Claude**, highlighting its capabilities, models, and commitment to responsible AI development. Key elements include:

## Claude Overview
- Claude is designed for safety and human benefit, emphasizing responsible AI technology.
- The latest model, **Claude Opus 4**, is now available, along with other models such as **Claude Sonnet 4** and **Claude Haiku 3.5**.

## Features
- The AI offers various plans (Max, Team, Enterprise) and downloadable apps.
- Users can engage with Claude and utilize the API to build AI-powered applications.

## Research & Initiatives
- Anthropic conducts extensive research on AI safety and its societal impacts, with a focus on long-term well-being.
- The site outlines the company's **Responsible Scaling Policy** and emphasizes transparency in AI development.

## News & Announcements
- Recent announcements include the launch of Claude Opus 4 and ongoing updates on AI safety issues.

## Education & Resources
- The **Anthropic Academy** provides learning resources for users to understand and build with Claude.

## Company Insights
- Anthropic highlights its ethics in AI development and provides information about career opportunities, events, and policy initiatives.

The site serves as a comprehensive resource for users interested in AI development, integration, and the responsible use of technology.